In [ ]:
from nltk.lm import Vocabulary

%load_ext autoreload
%autoreload 2

import os
from email import policy
from email.parser import BytesParser

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl

from src.spam_classifier.constants import PROJECT_ROOT

DATA_DIR = PROJECT_ROOT / "data"

In [ ]:
def parse_email(filepath):
    with open(filepath, "rb") as f:
        msg = BytesParser(policy=policy.default).parse(f)

    try:
        body = ""
        if msg.is_multipart():
            for part in msg.walk():
                if part.get_content_type() == "text/plain" and not part.get_content_disposition():
                    body += f"{part.get_content()} "
                    break
        else:
            body = msg.get_content()
    except Exception as e:
        print(f"Error processing {filepath.name}: {e}")

    return {"body": body, "type": msg.get_content_type()}

In [ ]:
for dir in (DATA_DIR).iterdir():
    if "ham" in dir.name:
        print(len(list(dir.iterdir())))

In [ ]:
ham_emails = []
spam_emails = []
for dir in (DATA_DIR).iterdir():
    if "spam" in dir.name:
        spam_emails += [parse_email(f) for f in dir.iterdir()]
    else:
        ham_emails += [parse_email(f) for f in dir.iterdir()]

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

from src.spam_classifier.mail_class import Mail


class MailTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        email_list = []
        for row in X.rows():
            body, type = row
            email_list.append(Mail(body, type).transform_mail())
        return email_list

In [ ]:
class MailVocabulary(BaseEstimator, TransformerMixin):
    def __init__(self, vocab_size: int = 1000):
        self.vocab_size = vocab_size

    def fit(self, X, y=None):
        word_counter = {}
        for word_dict in X:
            for word, count in word_dict.items():
                word_counter[word] = word_counter.get(word, 0) + count
        most_common = list(
            dict(sorted(word_counter.items(), key=lambda x: x[1], reverse=True)).keys()
        )[: self.vocab_size]
        self.vocabulary_ = {word: i for i, word in enumerate(["unknown"] + most_common)}
        return self

    def transform(self, X):
        transformed_X = []
        for word_dict in X:
            email_dict = [
                word_dict[count] for _, count in zip(self.vocabulary_.keys(), word_dict.keys())
            ]
            transformed_X.append(email_dict)

        return transformed_X

In [ ]:
X = ham_emails + spam_emails
y = [0 if email in ham_emails else 1 for email in X]

In [ ]:
from sklearn.model_selection import train_test_split

random_state = 42

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=random_state
)

X_val, X_test, y_val, y_test = train_test_split(
    X_test, y_test, test_size=0.5, stratify=y_test, random_state=random_state
)